# Smoothed Opacities

In this notebook we will smooth single grain opacities over a small distribution of similar sized particles to avoid strong, unrealistic resonances.

Since linear averaging produces smoother results, we will use those as standard.

In [ ]:
import os
import pickle
import shutil
import warnings

import numpy as np
import matplotlib.pyplot as plt

import dsharp_opac as opacity

import __main__ as main
interactive = not hasattr(main, '__file__')
if interactive:
    from IPython import get_ipython
    get_ipython().magic('matplotlib inline')
    
plt.style.use([{'figure.dpi':150}])

Define the size and wavelength grids

In [ ]:
a = np.logspace(-5, 2, 200)
lam = np.logspace(np.log10(1e-5), 1, 210)

Set wheter large grains should be extrapolated (large speed-up, but less precise) or not (very slow, but more precise)

In [ ]:
extrapolate_large_grains = False
fm_ice = 0.2
output_dir = 'data'

In [ ]:
if fm_ice == 0.2:
    name = 'default_opacities'
elif fm_ice == 0.0:
    name = 'icefree_opacities'
else:
    warnings.warn('fm_ice should be 0.0 or 0.2')
    name = 'other_opacities'
    
if extrapolate_large_grains:
    name += '_extrapol'
    
print('opacity name: {}'.format(name))
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

Calculate the current default opacities

In [ ]:
dc,rho_s = opacity.get_dsharp_mix(fm_ice=fm_ice)
res_default = opacity.get_opacities(a, lam, rho_s=rho_s, diel_const=dc, extrapolate_large_grains=extrapolate_large_grains)

k_abs  = res_default['k_abs']
k_sca  = res_default['k_sca']
g      = res_default['g']
theta  = res_default['theta']
S1     = res_default['S1']
S2     = res_default['S2']
rho_s  = res_default['rho_s']

Store the default opacities

In [ ]:
np.savez_compressed(
    os.path.join(output_dir,name),
    a     = res_default['a'],
    lam   = res_default['lam'],
    k_abs = res_default['k_abs'],
    k_sca = res_default['k_sca'],
    g     = res_default['g'],
    rho_s = res_default['rho_s'])

In [ ]:
with np.load(os.path.join(output_dir,name + '.npz')) as data:
    a     = data['a']
    lam   = data['lam']
    k_abs = data['k_abs']
    k_sca = data['k_sca']
    g     = data['g']
    rho_s = data['rho_s']

## Linear Averaging

In [ ]:
res_l = opacity.get_smooth_opacities(a, lam, rho_s, dc, smoothing='linear', extrapolate_large_grains=extrapolate_large_grains)

In [ ]:
with open('res_l.pickle', 'wb') as fid:
    pickle.dump(res_l, fid)

In [ ]:
with open('res_l.pickle', 'rb') as fid:
    res_l = pickle.load(fid)

In [ ]:
a_l         = res_l['a_h']
k_abs_l     = res_l['k_abs']
k_sca_l     = res_l['k_sca']
g_l         = res_l['g']
k_abs_avg_l = res_l['k_abs_avg_h']
k_sca_avg_l = res_l['k_sca_avg_h']
g_avg_l     = res_l['g_avg_h']

In [ ]:
np.savez_compressed(os.path.join(output_dir,name + '_smooth'),  a=a,   lam=lam, k_abs=k_abs_avg_l, k_sca=k_sca_avg_l, g=g_avg_l, rho_s=rho_s)
np.savez_compressed(os.path.join(output_dir,name + '_highres'), a=a_l, lam=lam, k_abs=k_abs_l,     k_sca=k_sca_l,     g=g_l,     rho_s=rho_s)

Plot the first part of the grid to make sure it looks correct (no double points and such).

In [ ]:
n_inter = 40

f, ax = plt.subplots(1, 2, figsize=(10, 3), tight_layout=True)
ax[0].plot(a[0:2], np.ones(2), 'x')
ax[0].plot(a_l[0:n_inter], np.ones(n_inter), 'r+')
ax[0].plot(a_l[n_inter:2 * n_inter], np.ones(n_inter), 'g+')
ax[0].set_xlim(a[0] - 1.1 * 0.5 * (a[1] - a[0]), a[1])

ax[1].plot(np.arange(0,n_inter),a_l[0:n_inter], 'x');
ax[1].plot(np.arange(n_inter,2 * n_inter), a_l[n_inter:2 * n_inter], 'x');

## Gaussian Averaging

In [ ]:
res_g = opacity.get_smooth_opacities(a, lam, rho_s, dc, smoothing='gaussian', extrapolate_large_grains=extrapolate_large_grains)

In [ ]:
with open('res_g.pickle', 'wb') as fid:
    pickle.dump(res_g, fid)

In [ ]:
with open('res_g.pickle', 'rb') as fid:
    res_g = pickle.load(fid)

In [ ]:
a_g         = res_g['a_h']
k_abs_g     = res_g['k_abs']
k_sca_g     = res_g['k_sca']
g_g         = res_g['g']
k_abs_avg_g = res_g['k_abs_avg_h']
k_sca_avg_g = res_g['k_sca_avg_h']
g_avg_g     = res_g['g_avg_h']

Plot the first part of the grid to make sure it looks correct (no double points and such).

In [ ]:
n_inter = 40

f, ax = plt.subplots(1, 2, figsize=(10, 3), tight_layout=True)
ax[0].plot(a[0:2], np.ones(2), 'x', markersize=10)
for i in range(3):
    ax[0].semilogx(a_g[i * n_inter: (i + 1) * n_inter], np.ones(n_inter), '+')
ax[0].set_xlim(a[0] - 1.1 * 0.5 * (a[1] - a[0]), a[i] + 0.5 * (a[i + 1] - a[i]))

ax[1].plot(np.arange(0,n_inter),a_g[0:n_inter], 'x');
ax[1].plot(np.arange(n_inter,2 * n_inter), a_g[n_inter:2 * n_inter], 'x');

## Plotting

Show the size averaged opacity and $\beta$ as function of $a_\mathrm{max}$.

In [ ]:
lam_avg =[0.1, 0.3]

f, axs = plt.subplots(2,1,sharex=True, figsize=(6,6))

r = opacity.size_average_opacity(lam_avg, a, lam, k_abs, k_sca, plot=True, ax=axs)
for i, _lam in enumerate(lam_avg):
    print('kappa_abs @ {:7.3g} mm for a_max = 1mm : {:.3g} cm^2/g'.format(_lam *10, np.interp(_lam, a, r['ka'][i])))

rl = opacity.size_average_opacity(lam_avg, a, lam, k_abs_avg_l, k_sca_avg_l, plot=False)
r['ax2'].semilogx(a, rl['beta'],'--',label='linear')

rg = opacity.size_average_opacity(lam_avg, a, lam, k_abs_avg_g, k_sca_avg_g, plot=False)
r['ax2'].semilogx(a, rg['beta'],'-.',label='Gaussian')
r['ax2'].legend()

Plot the opacities averaged over a typical size distribution to check if the results got smoother

In [ ]:
# a 1g-normalized size distribution (bin-integrated) up to 1 mm

s = a**0.5
s[a > 0.1] = 0
s= s / s.sum()

# size average the absorption opacities

ka0 = (k_abs * s[:,None]).sum(0)
kal = (k_abs_avg_l * s[:,None]).sum(0)
kag = (k_abs_avg_g * s[:,None]).sum(0)


# where to measure the reference value
lam_obs = 0.087

# load the D'Alessio opacity
d2g = sum([0.0056, 0.0034, 0.000768, 0.0041])
data_d01 = np.loadtxt(opacity.get_datafile('kappa_D01_T100K_p3.5_amax1mm.csv'))
lam_d01 = 10.**data_d01[:,0] * 1e-4
kap_d01 = 10.**(data_d01[:,1]) / d2g

# the Beckwith 1990 law

kb = 3.5 * (lam / 0.087)**(-1)  # Beckwith 1990

# the opacities from Andrews et al. 2009

la, ka = np.loadtxt(opacity.get_datafile('andrews2009.dat')).T

# now the plot

f, ax = plt.subplots(figsize=(7,7))

ax.plot(np.log10(1e4*lam),     np.log10(kb),      'k--', zorder=-100, alpha=0.5, label='Beckwith et al. 1990, $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(     lam_obs*1e4, np.interp(lam_obs,lam,kb)))
ax.plot(np.log10(la),          np.log10(ka),      'k--', zorder=-100, alpha=1.0, label='Andrews et al. 2009, $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(      lam_obs*1e4, np.interp(lam_obs*1e4,la,ka)))
ax.plot(np.log10(1e4*lam_d01), np.log10(kap_d01), 'k:',  zorder=-100, alpha=1.0, label='D\'Alessio et al. 2001, $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(   lam_obs*1e4, np.interp(lam_obs,lam_d01,kap_d01)))
ax.plot(np.log10(1e4*lam),     np.log10(ka0),            zorder=-100, lw=1, alpha=1.0, label='DSHARP (non-avg), $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(         lam_obs*1e4, np.interp(lam_obs,lam,ka0)))
ax.plot(np.log10(1e4*lam),     np.log10(kal),            zorder=-100, lw=1, alpha=1.0, label='DSHARP (linear-avg), $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(      lam_obs*1e4, np.interp(lam_obs,lam,kal)))
ax.plot(np.log10(1e4*lam),     np.log10(kag),            zorder=-100, lw=1, alpha=1.0, label='DSHARP (gauss-avg), $\kappa_{{{:.0f}}} = {:3.2g}$ cm$^2$/g'.format(       lam_obs*1e4, np.interp(lam_obs,lam,kag)))

ax.legend(loc=3, fontsize='small')
ax.set_xlim(-1,4)
ax.set_ylim(-1,5)
ax.set_xlabel('log $\lambda$ [$\mu$m]')
ax.set_ylabel('log $\kappa$ (dust) [cm$^2$/g]')
plt.savefig('avgs.pdf')

Make a movie scrolling through all wavelengths to show how much the fits differ.

In [ ]:
moviename = 'movie01.mp4'

if not os.path.isdir('imgs'):
    os.mkdir('imgs')

if os.path.isfile(moviename):
    os.unlink(moviename)
    
with plt.rc_context(rc={'lines.linewidth': 1}):
    f, axs = plt.subplots(2, 1, figsize=(15,10), sharex=True)
    
    for ilam in range(len(lam)):
        if ilam==0:
            l01, = axs[0].loglog(a, k_abs[:, ilam], 'k', label='original')
            l02, = axs[0].loglog(a_l, k_abs_l[:, ilam],'-', lw=1, c='0.75', label='highres')
            l03, = axs[0].loglog(a, k_abs_avg_l[:, ilam], label='linear-averaged')
            l04, = axs[0].loglog(a, k_abs_avg_g[:, ilam], label='gauss-averaged')
            leg=axs[0].legend()
            leg.set_title('absorption opacity', {'size':'large'})

            l11, = axs[1].loglog(a, k_sca[:, ilam], 'k', label='original')
            l12, = axs[1].loglog(a_l, k_sca_l[:, ilam],'-', lw=1, c='0.75', label='highres')
            l13, = axs[1].loglog(a, k_sca_avg_l[:, ilam], label='linear-averaged')
            l14, = axs[1].loglog(a, k_sca_avg_g[:, ilam], label='gauss-averaged')
            leg = axs[1].legend()
            leg.set_title('scattering opacity', {'size':'large'})

            for ax in axs:
                ax.set_xlim(1e-4,3e0)
                ax.set_ylim(ymin=2e-3, ymax=1e3)
                ax.set_xlabel('particle radius [cm]')

            axs[0].set_ylabel('absorption opacity [cm]')
            axs[1].set_ylabel('scattering opacity [cm]')
            ti = axs[0].set_title(f'wave length = {lam[ilam]:3.2} cm')
            f.subplots_adjust(hspace=0)
        else:
            l01.set_ydata(k_abs[:, ilam])
            l02.set_ydata(k_abs_l[:, ilam])
            l03.set_ydata(k_abs_avg_l[:, ilam])
            l04.set_ydata(k_abs_avg_g[:, ilam])
            
            l11.set_ydata(k_sca[:, ilam])
            l12.set_ydata(k_sca_l[:, ilam])
            l13.set_ydata(k_sca_avg_l[:, ilam])
            l14.set_ydata(k_sca_avg_g[:, ilam])
            
            ti.set_text(f'wave length = {lam[ilam]:3.2} cm')
        f.savefig(f'imgs/img_{ilam:03d}.png',dpi=200)

if os.system('ffmpeg -r 60 -f image2 -i imgs/img_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p ' + moviename) == 0:
    shutil.rmtree('imgs')
    print('finished making movie')
else:
    if shutil.which('ffmpeg') is None:
        print('ffmpeg not available, keeping the frames in folder \'img\'')
    else:    
        print('making movie failed, will not delete the frames')